In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf
from datetime import datetime
from sklearn.cluster import DBSCAN
from collections import Counter
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize']= 20,20
%matplotlib inline
import numpy as np
import pandas as pd
 
 

In [2]:
spark = SparkSession \
.builder \
.appName("GPS Spark") \
.getOrCreate()

In [3]:
s=spark.sparkContext

In [4]:
#points received on one day
sql_c=SQLContext(s)
df1= sql_c.read.orc("D:/data processing fyp/date=2019-12-12/part-00000-800de0ad-17a2-49c7-a953-f7c08c35735a.c000.snappy.orc")
df2= sql_c.read.orc("D:/data processing fyp/date=2019-12-12/part-00001-800de0ad-17a2-49c7-a953-f7c08c35735a.c000.snappy.orc")
df3= sql_c.read.orc("D:/data processing fyp/date=2019-12-12/part-00002-800de0ad-17a2-49c7-a953-f7c08c35735a.c000.snappy.orc")
df=df1.union(df2).union(df3)
 
   
 

In [5]:
df.show(5)

+--------------------+------+----------+-------------+-------------+-------------+
|           driver_id|status| timestamp|     latitude|    longitude|vehicle_model|
+--------------------+------+----------+-------------+-------------+-------------+
|d7a51d612d9293a49...|     F|1576111862|6.83232975006|79.9051895142|            4|
|f1e22a96f4fd25deb...|     F|1576109460|6.95281648636|79.9979705811|            1|
|b28002cd666068ae7...|     F|1576112077|6.85527467728| 79.890914917|            1|
|1626f7e9f29838853...|     B|1576109783|7.00499725342|79.8997268677|            1|
|620e1725123b5524b...|     F|1576111937|6.93429756165|79.8507766724|            2|
+--------------------+------+----------+-------------+-------------+-------------+
only showing top 5 rows



In [6]:
 df.count() #one day points count

30718021

In [7]:
tp1= sql_c.read.orc("D:/data processing fyp/dropdatesk=20191212/part-00000-bb193ffb-e8e1-4c1a-8898-5b1888d152f5.c000.snappy.orc")
tp2= sql_c.read.orc("D:/data processing fyp/dropdatesk=20191212/part-00001-bb193ffb-e8e1-4c1a-8898-5b1888d152f5.c000.snappy.orc")
tp3= sql_c.read.orc("D:/data processing fyp/dropdatesk=20191212/part-00002-bb193ffb-e8e1-4c1a-8898-5b1888d152f5.c000.snappy.orc")
tp4= sql_c.read.orc("D:/data processing fyp/dropdatesk=20191212/part-00003-bb193ffb-e8e1-4c1a-8898-5b1888d152f5.c000.snappy.orc")
tp5= sql_c.read.orc("D:/data processing fyp/dropdatesk=20191212/part-00004-bb193ffb-e8e1-4c1a-8898-5b1888d152f5.c000.snappy.orc")
tp6= sql_c.read.orc("D:/data processing fyp/dropdatesk=20191212/part-00005-bb193ffb-e8e1-4c1a-8898-5b1888d152f5.c000.snappy.orc")
tp7= sql_c.read.orc("D:/data processing fyp/dropdatesk=20191212/part-00006-bb193ffb-e8e1-4c1a-8898-5b1888d152f5.c000.snappy.orc")
tp8= sql_c.read.orc("D:/data processing fyp/dropdatesk=20191212/part-00007-bb193ffb-e8e1-4c1a-8898-5b1888d152f5.c000.snappy.orc")
tp9= sql_c.read.orc("D:/data processing fyp/dropdatesk=20191212/part-00008-bb193ffb-e8e1-4c1a-8898-5b1888d152f5.c000.snappy.orc")
tp10= sql_c.read.orc("D:/data processing fyp/dropdatesk=20191212/part-00009-bb193ffb-e8e1-4c1a-8898-5b1888d152f5.c000.snappy.orc")
trip=tp1.union(tp2).union(tp3).union(tp4).union(tp5).union(tp6).union(tp7).union(tp8).union(tp9).union(tp10)




In [8]:
trip.count()

80775

In [9]:
#get the distinct drivers from a particular driver
drivers=df.groupBy('driver_id').count()
list=drivers.rdd.map(lambda x: x[0]).collect()

In [10]:
def getDriverTripDF(driverid):
    driver_td_df=trip.select(["tripid","driverid","pickuplatitude","pickuplongitude","droplatitude","droplongitude", "actualpickuptime","droptime"]).where(trip.driverid == driverid ).sort("tripid").collect()
    columns=["tripid","driverid","pickuplatitude","pickuplongitude","droplatitude","droplongitude", "actualpickuptime","droptime"]
    df_trip_driver = pd.DataFrame(driver_td_df, columns=columns)
    
    return df_trip_driver
    

In [11]:
def getDriverGPSpoints(driverid):
    driver_gps_df=df.select(["driver_id","timestamp","latitude","longitude"]).where(df.driver_id == driverid ).sort("timestamp").collect()    
    columns2=["driver_id","timestamp","latitude","longitude"]
    df_gps_driver = pd.DataFrame(driver_gps_df, columns=columns2)
    return df_gps_driver
    

In [12]:
df_trip_driver=getDriverTripDF("4e1020dee5af4da54b0ef441f565971777785152292a1877a2e98a9c6e223ab4")
df_trip_driver

,tripid,driverid,pickuplatitude,pickuplongitude,droplatitude,droplongitude,actualpickuptime,droptime
0,198642054,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.8922200000,79.9049000000,6.8924600000,79.9044000000,2019-12-12 14:57:02,2019-12-12 15:23:47
1,198650722,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.8940100000,79.9064000000,6.9330500000,79.8562000000,2019-12-12 15:29:32,2019-12-12 16:01:38
2,198660380,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.9326900000,79.8568000000,6.9072000000,79.8523000000,2019-12-12 16:13:54,2019-12-12 16:27:35
3,198728862,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.9156500000,79.8548000000,6.8562900000,79.8852000000,2019-12-12 20:08:44,2019-12-12 20:37:12
4,198803270,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.8404500000,79.9287000000,6.8736500000,79.8899000000,2019-12-12 23:24:09,2019-12-13 00:35:56
5,198842574,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.8683900000,79.8864000000,6.8348200000,79.8727000000,2019-12-13 00:49:11,2019-12-13 01:16:09
6,198856294,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.8402700000,79.8695000000,6.8331500000,79.8624000000,2019-12-13 01:26:13,2019-12-13 01:30:25


In [13]:
for num in df_trip_driver.index:
        df_trip_driver.loc[num,'Actualduration'] = (df_trip_driver['droptime'][num]-df_trip_driver['actualpickuptime'][num]).total_seconds()
         

In [14]:
df_trip_driver

,tripid,driverid,pickuplatitude,pickuplongitude,droplatitude,droplongitude,actualpickuptime,droptime,Actualduration
0,198642054,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.8922200000,79.9049000000,6.8924600000,79.9044000000,2019-12-12 14:57:02,2019-12-12 15:23:47,1605.0
1,198650722,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.8940100000,79.9064000000,6.9330500000,79.8562000000,2019-12-12 15:29:32,2019-12-12 16:01:38,1926.0
2,198660380,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.9326900000,79.8568000000,6.9072000000,79.8523000000,2019-12-12 16:13:54,2019-12-12 16:27:35,821.0
3,198728862,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.9156500000,79.8548000000,6.8562900000,79.8852000000,2019-12-12 20:08:44,2019-12-12 20:37:12,1708.0
4,198803270,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.8404500000,79.9287000000,6.8736500000,79.8899000000,2019-12-12 23:24:09,2019-12-13 00:35:56,4307.0
5,198842574,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.8683900000,79.8864000000,6.8348200000,79.8727000000,2019-12-13 00:49:11,2019-12-13 01:16:09,1618.0
6,198856294,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.8402700000,79.8695000000,6.8331500000,79.8624000000,2019-12-13 01:26:13,2019-12-13 01:30:25,252.0


In [15]:
df_gps_driver=getDriverGPSpoints("4e1020dee5af4da54b0ef441f565971777785152292a1877a2e98a9c6e223ab4")
df_gps_driver

,driver_id,timestamp,latitude,longitude
0,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576113891,6.859347,79.926369
1,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576113901,6.858833,79.926437
2,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576113911,6.858485,79.926262
3,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576113921,6.858469,79.925667
4,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576113931,6.858158,79.925598
...,...,...,...,...
2155,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576172152,6.859390,79.925026
2156,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576172162,6.859390,79.925026
2157,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576172172,6.859390,79.925026
2158,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576172182,6.859428,79.924934


In [16]:
for num in df_gps_driver.index:
        df_gps_driver.loc[num,'time'] = datetime.fromtimestamp(df_gps_driver.loc[num,'timestamp'])
         

In [17]:
df_gps_driver

,driver_id,timestamp,latitude,longitude,time
0,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576113891,6.859347,79.926369,2019-12-12 06:54:51
1,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576113901,6.858833,79.926437,2019-12-12 06:55:01
2,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576113911,6.858485,79.926262,2019-12-12 06:55:11
3,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576113921,6.858469,79.925667,2019-12-12 06:55:21
4,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576113931,6.858158,79.925598,2019-12-12 06:55:31
...,...,...,...,...,...
2155,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576172152,6.859390,79.925026,2019-12-12 23:05:52
2156,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576172162,6.859390,79.925026,2019-12-12 23:06:02
2157,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576172172,6.859390,79.925026,2019-12-12 23:06:12
2158,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576172182,6.859428,79.924934,2019-12-12 23:06:22


In [18]:
#assigning relevant tripid to traces
def assign_trip_id(df_gps_driver,df_trip_driver):
    df_gps_driver['tripid']= ' '
    for index in df_gps_driver.index :
        for num in df_trip_driver.index:
            if df_trip_driver['actualpickuptime'][num] < df_gps_driver["time"][index] and df_gps_driver["time"][index]<df_trip_driver['droptime'][num]:
                df_gps_driver.loc[index, 'tripid'] =  df_trip_driver['tripid'][num]
    df_gps_driver.drop(df_gps_driver[df_gps_driver['tripid'] ==' '].index, inplace = True)
    return df_gps_driver

        

In [19]:
df_gps_driver=assign_trip_id(df_gps_driver,df_trip_driver)
df_gps_driver

,driver_id,timestamp,latitude,longitude,time,tripid
990,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576142825,6.872882,79.873886,2019-12-12 14:57:05,198642054
991,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576142835,6.872423,79.874519,2019-12-12 14:57:15,198642054
992,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576142845,6.872085,79.875328,2019-12-12 14:57:25,198642054
993,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576142855,6.871644,79.876045,2019-12-12 14:57:35,198642054
994,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576142865,6.871482,79.876190,2019-12-12 14:57:45,198642054
...,...,...,...,...,...,...
2057,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576163180,6.840068,79.895447,2019-12-12 20:36:20,198728862
2058,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576163200,6.839825,79.896355,2019-12-12 20:36:40,198728862
2059,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576163210,6.840035,79.896957,2019-12-12 20:36:50,198728862
2060,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576163220,6.840290,79.897499,2019-12-12 20:37:00,198728862


In [20]:
#form detail dataframe to check enough points per trip to consider
 
driver_data =df_gps_driver.groupby('tripid').size()
driver_data=driver_data.to_frame().reset_index()
driver_data['Actualduration']=""
for num in driver_data.index :
        for i in df_trip_driver.index:
            if driver_data['tripid'][num]== df_trip_driver['tripid'][i]:
                driver_data.loc[num,'Actualduration'] = df_trip_driver['Actualduration'][i]
driver_data.columns=["tripid","points","Actualduration"]
driver_data["AverageP"]= driver_data["Actualduration"]/driver_data["points"]
 

In [21]:
driver_data 

,tripid,points,Actualduration,AverageP
0,198642054,84,1605.0,19.107143
1,198728862,141,1708.0,12.113475


In [22]:
#distance calculation for a particular trip_id

#function to find the distance
def haversine_distance(lat1, lon1, lat2, lon2):
   r = 6371
   phi1 = np.radians(lat1)
   phi2 = np.radians(lat2)
   delta_phi = np.radians(lat2 - lat1)
   delta_lambda = np.radians(lon2 - lon1)
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
   return np.round(res, 2)

 
def distance_per_trip(tripid,singleTripdf):
    Distance=0
    for num in range (len(singleTripdf)-1):
        Distance_interval= haversine_distance(singleTripdf.loc[num,'latitude'],singleTripdf.loc[num,'longitude'],singleTripdf.loc[num+1,'latitude'],singleTripdf .loc[num+1,'longitude'])
        Distance += Distance_interval
    return Distance 
def duration_per_trip(tripid,singleTripdf):  
    duration=(singleTripdf.loc[len(singleTripdf)-1,'time']-singleTripdf.loc[0,'time']).total_seconds()
    return  duration

In [23]:
df_gps_driver.reset_index(drop=True)

,driver_id,timestamp,latitude,longitude,time,tripid
0,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576142825,6.872882,79.873886,2019-12-12 14:57:05,198642054
1,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576142835,6.872423,79.874519,2019-12-12 14:57:15,198642054
2,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576142845,6.872085,79.875328,2019-12-12 14:57:25,198642054
3,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576142855,6.871644,79.876045,2019-12-12 14:57:35,198642054
4,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576142865,6.871482,79.876190,2019-12-12 14:57:45,198642054
...,...,...,...,...,...,...
220,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576163180,6.840068,79.895447,2019-12-12 20:36:20,198728862
221,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576163200,6.839825,79.896355,2019-12-12 20:36:40,198728862
222,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576163210,6.840035,79.896957,2019-12-12 20:36:50,198728862
223,4e1020dee5af4da54b0ef441f565971777785152292a18...,1576163220,6.840290,79.897499,2019-12-12 20:37:00,198728862


In [24]:
single_trip=df_gps_driver.loc[df_gps_driver['tripid'] == 181792366]
single_trip=single_trip.reset_index(drop=True)
distance_per_trip(181792366,single_trip)

0

In [25]:
#distance and duration calculation for tripids
driver_data['distance']=''
for i in range (len(driver_data)):
    single_trip=df_gps_driver.loc[df_gps_driver['tripid'] == driver_data['tripid'][i]]
    single_trip=single_trip.reset_index()
    driver_data.loc[i,'distance']= distance_per_trip(driver_data['tripid'][i],single_trip)
    driver_data.loc[i,'duration']= duration_per_trip(driver_data['tripid'][i],single_trip)

In [26]:
driver_data

,tripid,points,Actualduration,AverageP,distance,duration
0,198642054,84,1605.0,19.107143,4.03,830.0
1,198728862,141,1708.0,12.113475,5.46,1698.0


In [27]:
#def function for a single driver 
def driver_singleday(driver_id):
    df_trip_driver=getDriverTripDF(driver_id)
    df_gps_driver=getDriverGPSpoints(driver_id) 
    for num in df_trip_driver.index:
        df_trip_driver.loc[num,'Actualduration'] = (df_trip_driver['droptime'][num]-df_trip_driver['actualpickuptime'][num]).total_seconds()
    for num in df_gps_driver.index:
        df_gps_driver.loc[num,'time'] = datetime.fromtimestamp(df_gps_driver.loc[num,'timestamp'])
    df_gps_driver['tripid']= ' '
    for index in df_gps_driver.index :
        for num in df_trip_driver.index:
            if df_trip_driver['actualpickuptime'][num] < df_gps_driver["time"][index] and df_gps_driver["time"][index]<df_trip_driver['droptime'][num]:
                df_gps_driver.loc[index, 'tripid'] =  df_trip_driver['tripid'][num]
    df_gps_driver.drop(df_gps_driver[df_gps_driver['tripid'] ==' '].index, inplace = True)
    driver_data =df_gps_driver.groupby('tripid').size()
    driver_data=driver_data.to_frame().reset_index()
    driver_data['Actualduration']=""
    for num in driver_data.index :
        for i in df_trip_driver.index:
            if driver_data['tripid'][num]== df_trip_driver['tripid'][i]:
                driver_data.loc[num,'Actualduration'] = df_trip_driver['Actualduration'][i]
    driver_data.columns=["tripid","points","Actualduration"]
    driver_data["AverageP"]= driver_data["Actualduration"]/driver_data["points"]
    driver_data['distance']=''
    for i in range (len(driver_data)):
        single_trip=df_gps_driver.loc[df_gps_driver['tripid'] == driver_data['tripid'][i]]
        single_trip=single_trip.reset_index()
        driver_data.loc[i,'distance']= distance_per_trip(driver_data['tripid'][i],single_trip)
        driver_data.loc[i,'duration']= duration_per_trip(driver_data['tripid'][i],single_trip)
    driver_data.insert(0, "driver_id", driver_id)
    return  driver_data
 


In [28]:
#get the distinct drivers from a particular driver
drivers=df.groupBy('driver_id').count()
list=drivers.rdd.map(lambda x: x[0]).collect()

In [29]:
list=sorted(list)

In [30]:
list

['...',
 ...]

In [37]:
full = pd.DataFrame(columns=['driver_id','tripid','points','Actualduration','AverageP','distance','duration'])
 
count=0   
for i in list[4000:5000]:
    count+=1
    f= [full,driver_singleday(i)]     
    full=pd.concat(f) 
    print(count)
full 

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


,driver_id,tripid,points,Actualduration,AverageP,distance,duration
0,419eef2bb93fe53b4c63770e8657224f8a26739d758ffb...,198738364,479,4953.0,10.340292,17.59,4780.0
0,41a5b79505d1aa0ecc57bd85c01b8510481afe7ef30777...,198786252,143,1433.0,10.020979,16.45,1420.0
1,41a5b79505d1aa0ecc57bd85c01b8510481afe7ef30777...,198806322,170,3595.0,21.147059,14.34,1753.0
0,41af114c3a32d8639ac801a702237608c0e7bd9ab531d7...,198762046,96,1612.0,16.791667,4.04,960.0
0,41bee82cc0b1bb23541f623a0f0377f187f572e361dfb8...,198706422,23,3659.0,159.086957,1.22,220.0
...,...,...,...,...,...,...,...
1,523879b3489d452e39aa5c5455c2195db55ba75c08f63b...,198722876,160,3901.0,24.38125,0.0,1600.0
0,523e0d654e8b4a60feb5472609a35d6eb3572ef6d2ce8d...,198608571,200,2004.0,10.02,9.01,1990.0
1,523e0d654e8b4a60feb5472609a35d6eb3572ef6d2ce8d...,198624345,7,1271.0,181.571429,0.15,1260.0
2,523e0d654e8b4a60feb5472609a35d6eb3572ef6d2ce8d...,198635671,74,746.0,10.081081,1.57,730.0


In [38]:
full.to_csv('date=2019-12-12-part3-4000-5000.csv', index=False)